In [1]:
  
# SKT Brain의 KoBERT, 학습 및 테스트 데이터셋만 따로 준비 = 카카오 브런치[text, label]
# SKT Brain github 주소는 다음과 같습니다. https://github.com/SKTBrain/KoBERT

!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2 # 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
!pip install torch

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
#!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup


     |████████████████████████████████| 46.9 MB 1.4 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 4.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595734 sha256=988f3b67f630174793b990d60bb55b64e1621861f80f5489cd4eb6770c7ce85d
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 4.1 MB/s 
     |████████████████████████████████| 769 kB 4.3 MB/s 
     |████████████████████████████████| 895 kB 18.1 MB/s 
     |████████████████████████████████| 3.0 MB 35.1 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-fhgnrhk1
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pi

In [2]:
# GPU 사용 시
device = torch.device("cuda:0")
#device = torch.device("cpu")

# BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

.cache/kobert_v1.zip[██████████████████████████████████████████████████]
.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [3]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 데이터셋 불러오기
import pandas as pd
df = pd.read_excel('/content/drive/Shareddrives/실전기계_project/data_labeling/data_labeling_3.xlsx', sheet_name=None)
df = pd.concat(df, ignore_index=True)
data = df[['document', 'spo_label']]

In [5]:
data['spo_label'].unique()

array([ 0.,  1., nan])

In [6]:
# column명 변경
data.rename(columns={'document':'content', 'spo_label':'label'}, inplace=True)

# type 변경
data = data.astype({'label':'object'})
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5251 entries, 0 to 5250
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  5251 non-null   object
 1   label    782 non-null    object
dtypes: object(2)
memory usage: 82.2+ KB


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [7]:
data = data.fillna(0)

In [8]:
data.head()

,content,label
0,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,0.0
1,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",0.0
2,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,0.0
3,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,0.0
4,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,0.0


In [9]:
data['label'].value_counts()

0.0    4624
1.0     627
Name: label, dtype: int64

In [10]:
df_0 = data[data['label'] == 0]
df_1 = data[data['label'] == 1]

In [11]:
df = pd.concat([df_0[:627], df_1])
df = df.sample(frac=1)

In [12]:
df['label'].value_counts()

0.0    627
1.0    627
Name: label, dtype: int64

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1254 entries, 2882 to 443
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   content  1254 non-null   object 
 1   label    1254 non-null   float64
dtypes: float64(1), object(1)
memory usage: 29.4+ KB


In [14]:
df = df.astype({'label':'int'})
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1254 entries, 2882 to 443
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  1254 non-null   object
 1   label    1254 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 29.4+ KB


In [15]:
new_data = df
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1254 entries, 2882 to 443
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  1254 non-null   object
 1   label    1254 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 29.4+ KB


In [16]:
# label encoding
new_data.loc[(new_data['label'] == "0"), 'label'] = 0  # spoX => 0
new_data.loc[(new_data['label'] == "1"), 'label'] = 1  # spo => 1


# list형태로 데이터 저장 -> [content, label] 형태
data_list = []
for q, label in zip(new_data['content'], new_data['label'])  :
    data = []
    data.append(str(q)) # str타입으로 변환, 안 해주면 나중에 타입변환할 때 오류남
    data.append(str(label))

    data_list.append(data)

In [17]:
# 데이터 확인
print(data_list[0])
print(data_list[-1])

['스텝업 1과 2의 남주 둘이서 여자 하나 놓고 싸우네요...ㅋㅋㅋ 재밌어요', '1']
['이스트우드 최고작', '0']


In [18]:
# train, test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=42)

In [19]:
# 데이터 개수 확인
print(len(dataset_train))
print(len(dataset_test))

1003
251


In [20]:
# KoBERT모델의 입력으로 들어갈 수 있는 형태가 되도록 변환해주는 class
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        # BERTSentenceTransform이라는 모듈 사용
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        # 타입 변환
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    # item 가져오기
    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    # 길이 출력
    def __len__(self):
        return (len(self.labels))

In [21]:
# Setting parameters
max_len = 32
batch_size = 16
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [22]:
# 토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# 패딩
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model


In [23]:
# 데이터 확인 -> array1은 시퀀스, array2는 길이와 타입, array3은 attention mask 시퀀스(1로 패딩된 값들은 연산할 필요가 없기 때문에, 연산을 하지 않아도 됨 -> 이걸 알려주는 데이터가 바로 얘)
data_train[0]

(array([   2, 2415, 7096, 4117, 6361, 5872, 6903, 7095, 2649, 7088, 2767,
        7870, 3909, 7141, 5400, 1958, 7794, 5140, 5938, 2358,  517, 6172,
        6179, 7767, 5330, 1258, 5330, 6149, 7997, 5943, 7096,    3],
       dtype=int32),
 array(32, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
 1)

In [24]:
# torch 형식의 dataset 생성
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [25]:
# KoBERT 학습모델
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   # 클래스 수 조정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [26]:
# BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

# optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio) # 초기에 성능을 끌어올리는 것 = 운동할 때 워밍업 느낌

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0] # 맞은 비율
    return train_acc
    
train_dataloader

In [27]:
import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 모델 학습시키기
for e in range(3):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/63 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.7087417244911194 train acc 0.5
epoch 1 train acc 0.5969516594516595


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 2 test acc 0.7599431818181818


  0%|          | 0/63 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.4770071804523468 train acc 0.8125
epoch 3 train acc 0.7882395382395382


  0%|          | 0/16 [00:00<?, ?it/s]

epoch 3 test acc 0.6299715909090909


In [28]:
import numpy as np
import matplotlib.pyplot as plt
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [29]:
# 토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# 새로운 문장 테스트
def predict(predict_sentence):
  data = [predict_sentence, '0']
  dataset_another = [data]

  another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
      token_ids = token_ids.long().to(device)
      #print("token_ids : ", token_ids)
      segment_ids = segment_ids.long().to(device)
      #print("segment_ids : ", segment_ids)

      valid_length= valid_length
      #print("valid_length : ", valid_length)
      label = label.long().to(device)
      #print("label : ", label)
      out = model(token_ids, valid_length, segment_ids)

      #print("out : ", out)

      test_eval=[]
      for i in out:
          logits=i
          logits = logits.detach().cpu().numpy()
          logits = softmax(logits)
          print("softmax : ", logits)

          if np.argmax(logits) == 0:
                test_eval.append("스포x")
          elif np.argmax(logits) == 1:
                test_eval.append("스포")
        

      print(">> 이 리뷰는 " + test_eval[0] + " 입니다.")

using cached model


In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("리뷰를 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

리뷰를 입력해주세요 : 주인공 죽음


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


softmax :  [0.80435425 0.1956458 ]
>> 이 리뷰는 스포x 입니다.


리뷰를 입력해주세요 : 반전
softmax :  [0.80786425 0.19213577]
>> 이 리뷰는 스포x 입니다.




In [ ]:
torch.save(model.state_dict(), '/content/drive/Shareddrives/실전기계_project/model_state_dict.pt')

In [ ]:
torch.save(model, '/content/drive/Shareddrives/실전기계_project/model.pt')